In [1]:
import pywhatkit as kit
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import time
import pandas as pd
import pyautogui
import os
import shutil

In [ ]:
#En este archivo están los clientes con sus números celulares registrados para notificar
con_suministro = pd.read_excel(r'-df_con_suministro_3.xlsx')


con_suministro["TELEFONO PAGOS"] =  con_suministro["TELEFONO PAGOS"].astype(float)
con_suministro["TELEFONO PAGOS"] =  con_suministro["TELEFONO PAGOS"].apply(lambda x: f'{int(x)}')
con_suministro["TELEFONO PAGOS"] =  con_suministro["TELEFONO PAGOS"].astype(str)


def guardar_registro(numero, mensaje, enviado_exitosamente):
    with open("registro_mensajes.txt", "a") as file:
        if enviado_exitosamente:
            file.write(f"Mensaje enviado a {numero}: {mensaje} - EXITOSO\n")
        else:
            file.write(f"Mensaje a {numero} ({mensaje}) - FALLIDO\n")

df_reporte = pd.DataFrame()
data_list = []            
 
for index, row in con_suministro.iterrows():
    x = datetime.now().hour
    y = +datetime.now().minute + 1
 
    numero = str(row["TELEFONO PAGOS"])  
    suministro = str(row['Suministro'])
    deuda = str(row['DEUDA TOTAL'])  
    direccion = row['DIRECCION']
    antigüedad = str(row['ANTIGUEDAD SIMULADA'])
    distrito =  row['DISTRITO']
 
    
    mensaje = f"*¡Atención! Luz del Sur le informa que su suministro tiene un corte por deuda.* \nEstimado cliente, el suministro {suministro} se encuentra en SITUACION DE CORTE, debido a que mantiene {antigüedad} facturaciones pendientes de pago por S/ {deuda}, adicional a ello se vienen generado cargos por intereses y moras. Para evitar el corte del servicio, deberá realizar el pago a través de nuestra Página Web WWW.LUZDELSUR.COM.PE, App de Luz del Sur o acercándose a nuestras oficinas de atención al público. En caso de haber cancelado agradeceremos dejar sin efecto el presente mensaje."
    try:
        # Enviar el mensaje usando pywhatkit
        kit.sendwhatmsg(f"+{numero}", mensaje, x, y)  # El tercer argumento es la hora y el cuarto es los minutos
        print(f"Mensaje enviado a {numero}: {mensaje}")
        enviado_exitosamente = True
    except Exception as e:
        print(f"No se pudo enviar el mensaje a {numero}: {mensaje}")
        enviado_exitosamente = False

    row_dict = {'SUMINISTRO': suministro, 'CELULAR': numero, 'RESULTADO': enviado_exitosamente, 'MENSAJE': mensaje}
    data_list.append(row_dict)    
    
    guardar_registro(numero,mensaje,enviado_exitosamente)
    
    time.sleep(4)
    
    print(pyautogui.position())
     
    pyautogui.click(x=1805, y=969)
    time.sleep(3)
    pyautogui.click(x=1805, y=969)
    time.sleep(3)
    pyautogui.click(x=880, y=26)
    time.sleep(2)
    pyautogui.click(x=1039, y=296) 

In [4]:
nombre_archivo = "registro_mensajes.txt"

with open(nombre_archivo, 'r') as archivo:
    lineas = archivo.readlines()
data = {'Texto': lineas} 
df = pd.DataFrame(data)
df
def obtener_ultimas_siete_letras(texto):
    return texto[-8:]

df['Ultimas_siete_letras'] = df['Texto'].apply(obtener_ultimas_siete_letras)
hora_actual = datetime.now().strftime('%H-%M-%S')
nombre_archivo = f'reporte enviados{hora_actual}.xlsx'
df.to_excel(nombre_archivo, index=False)
